In [1]:
import csv
import os
import pandas as pd
import collections
import ast
import time
import collections
import json
import sqlite3
import xxhash
import numpy as np
import pickle 
import random
import glob
from io import StringIO
from pathlib import Path
from openai import OpenAI
import openai
print(openai.__version__)
os.environ["OPENAI_API_KEY"] = ''

client = OpenAI(
    # defaults to os.environ.get("OPENAI_API_KEY")
    api_key=os.getenv("OPENAI_API_KEY"),
)


1.35.10


In [2]:
import time as time
from multiprocessing import Process, Queue
import query_module
def evalfunc(sql_source, sql_target, db_path_source, db_path_target):
    if not os.path.isfile(db_path_source):
        print("cannot find file", db_path_source)
        return False
    if not os.path.isfile(db_path_target):
        print("cannot find file", db_path_target)
        return False
    timeout = 120
    output = Queue()
    query_process = Process(target=query_module.execute_query, args=(db_path_source, sql_source, output))
    query_process.start()
    output_hash = ''
    
    try:
        source_results = None
        source_results = output.get(True, timeout+5)
        query_process.join(timeout)
        if query_process.is_alive():
            print("process terminated")
            query_process.terminate()  # Terminate the process
            query_process.join()  # Make sure it's cleaned up
            return False, [Exception('SQL query took too much time to execute.')]
        if isinstance(source_results, Exception):
            raise source_results
        output_hash = xxhash.xxh128_hexdigest(str(len(source_results)), seed=123)
        connection = sqlite3.connect(db_path_target)
        cursor = connection.cursor()
        target_results = cursor.execute(sql_target).fetchall()
        cursor.close()
        connection.close()
        if len(source_results) != len(target_results):
            return False, []
        if 'ORDER BY' in sql_target:
            for a, b in zip(source_results, target_results):
                lhs = tuple(sorted(list(a), key=lambda x: hash(x)))
                rhs = tuple(sorted(list(b), key=lambda x: hash(x)))
                output_hash = xxhash.xxh128_hexdigest(output_hash + str(lhs), seed=123)
                if lhs != rhs:
                    return False, []
        else:
            lset, rset = set(), set()
            for a, b in zip(source_results, target_results):
                lset.add(tuple(sorted(list(a), key=lambda x: hash(x))))
                rset.add(tuple(sorted(list(b), key=lambda x: hash(x))))
            output_hash = xxhash.xxh128_hexdigest(str(lset), seed=123)
            if lset != rset:
                return False, []
    except Exception as ex:
        print(ex)
        return False, [ex]
    return True, []


In [3]:
def execute(sql, db_path):
    if not os.path.isfile(db_path):
        print("cannot find file")
        return False
    results = ''
    try:
        connection = sqlite3.connect(db_path, uri=True)
        cursor = connection.cursor()
        results = cursor.execute(sql).fetchall()
    except Exception as ex:
        cursor.close()
        connection.close()
        print(ex)
        return False
    finally:
        cursor.close()
        connection.close()
    return results

In [4]:
# all the key used
prefix_keys = {}
prefix_keys = pickle.load(open("prefix_keys.p", "rb"))

# Califronia Schools

In [5]:
from SQLs import create_california_llm_table, california_llm_table_insert_records
database = 'california_schools'
db_path = f'./../databases/dev_databases/{database}/{database}.sqlite'

connection = sqlite3.connect(db_path)
cursor = connection.cursor()
cursor.execute('DROP TABLE IF EXISTS llm;')
connection.commit()
connection.close()

connection = sqlite3.connect(db_path)
cursor = connection.cursor()
cursor.execute(create_california_llm_table)
connection.commit()
connection.close()

# modify
output_name = './llm_generations/5-shots/california_schools_gpt4turbo_5shot.jsonl'


rows = []
identifier_2_caschools_record = {}
with open(output_name, 'r') as outfile:
    for line in outfile:
        response = json.loads(line)
        rid = response['custom_id']
        result = response['response']['body']["choices"][0]['message']['content']
        result = result.replace(", '", ",'")
        result = result.replace("'s", '\"s')
        data_io = StringIO(result)
        reader = csv.reader(data_io, delimiter=',', quotechar="'")
        if 'fprm' in rid:
            fprm_entry = next(reader)
            fprm_entry = [str(e.strip()) for e in fprm_entry]
            if len(fprm_entry) < 5:
                fprm_entry = prefix_keys[rid] + fprm_entry
            elif len(fprm_entry)>5:
                num_to_concat = len(fprm_entry) - 5 + 1
                concatenated_value = ' '.join(fprm_entry[:num_to_concat])
                fprm_entry = [concatenated_value] + fprm_entry[num_to_concat:]
            if len(fprm_entry) != 5:
                continue
            # make sure keys are the same
            for i, key in enumerate(prefix_keys[rid]):
                fprm_entry[i] = key
            for i in range(7):
                fprm_entry.append(None)
            rows.append(tuple(fprm_entry))
        else:
            school_entry = next(reader)
            school_entry = [str(e.strip()) for e in school_entry]
            if len(school_entry) < 7:
                school_entry = prefix_keys[rid] + school_entry
            elif len(school_entry) > 7:
                num_to_concat = len(school_entry) - 7 + 1
                concatenated_value = ' '.join(school_entry[:num_to_concat])
                school_entry = [concatenated_value] + school_entry[num_to_concat:]
            if len(school_entry) != 7:
                continue
            for i, key in enumerate(prefix_keys[rid]):
                school_entry[i] = key
            temp = [None for i in range(5)]
            rows.append(tuple(temp+school_entry))
        identifier_2_caschools_record[rid] = rows[-1]

connection = sqlite3.connect(db_path)
cursor = connection.cursor()
cursor.executemany(california_llm_table_insert_records, rows)
connection.commit()
connection.close()

file_path = './../beyond-database-questions/CASchoolQueries.csv'
df = pd.read_csv(file_path, header=None)
correct = 30
index = 0
for sql1, sql2 in zip(df.iloc[:, 3], df.iloc[:, 5]):
    match, exception = evalfunc(sql1, sql2, db_path, db_path)
    if match != True:
        correct -= 1
    index += 1
print(correct, correct/30)

17 0.5666666666666667


In [6]:
wrong_cell_values = 0
total_cells = 0
database = 'california_schools'
# this need to be the db_path to the original db
db_path = f'./../databases/dev_databases/{database}/{database}.sqlite'

index = 0
for entry in execute('''
        SELECT `School Name`, `District Name`, `Educational Option Type`, `Charter School (Y/N)`, `Charter Funding Type` FROM frpm;
        ''', db_path):
    school = entry[0]
    district = entry[1]
    identifier = 'fprm-'+str(index)
    total_cells += 3
    if identifier not in identifier_2_caschools_record:
        wrong_cell_values += 3
        continue
    generated_records = identifier_2_caschools_record[identifier]
    for x, y in zip(entry[2:], generated_records[2:5]):
        if x != y:
            wrong_cell_values += 1
    index += 1

index = 0
for entry in execute('''
    SELECT Street, School, City, Magnet, Website, 
        CASE 
            WHEN Virtual = 'F' THEN 'Y' 
            ELSE 'N'
        END AS ExclusivelyVirtual,
        County
        FROM Schools 
    ''', db_path):
    street = entry[0]
    school = entry[1]
    identifier = 'schools-'+str(index)
    total_cells += 5
    if identifier not in identifier_2_caschools_record:
        wrong_cell_values += 5
        continue
    generated_records = identifier_2_caschools_record[identifier]
    for x, y in zip(entry[2:], generated_records[7:]):
        if x != y:
            wrong_cell_values += 1
    index += 1

print(total_cells - wrong_cell_values)
print(total_cells)

8984
37392


# Superhero

In [7]:
from SQLs import create_superhero_llm_table, create_superhero_llm_hero_2_power_table,\
    superhero_llm_insert_records, llm_hero_2_power_insert_records 

database = 'superhero'
db_path = f'./../databases/dev_databases/{database}/{database}.sqlite'
connection = sqlite3.connect(db_path)
cursor = connection.cursor()
cursor.execute('DROP TABLE IF EXISTS llm;')
cursor.execute('DROP TABLE IF EXISTS llm_hero_2_power;')
connection.commit()
connection.close()

#"superhero_name", "full_name", "eye_color", "hair_color", "skin_color", "publisher_name", "race", "gender", "moral_alignment"
data1 = []
# "superhero_name", "full_name", one "power_name"
data2 = []


# modify
output_name = 'llm_generations/5-shots/superhero_gpt4turbo_5shot.jsonl'


identifier_2_hero_record = {}
rows = []
with open(output_name, 'r') as outfile:
    for line in outfile:
        response = json.loads(line)
        rid = response['custom_id']
        result = response['response']['body']["choices"][0]['message']['content']
        result = result.replace(", '", ",'")
        result = result.replace("[", '')
        result = result.replace("]", '')
        data_io = StringIO(result)
        reader = csv.reader(data_io, delimiter=',', quotechar="'")
        entry = next(reader)
        entry = [e.strip() for e in entry]
        assert len(entry) >= 10, print(rid,entry)
        for i, key in enumerate(prefix_keys[rid]):
            entry[i] = key
        data1.append(entry[:9].copy())
        if len(entry) == 10:
            for power_name in entry[-1].split(','):
                data2.append([entry[0], entry[1], power_name])
        else:
            for power_name in entry[9:]:
                data2.append([entry[0], entry[1], power_name])
        identifier_2_hero_record[rid] = entry

connection = sqlite3.connect(db_path)
cursor = connection.cursor()
cursor.execute(create_superhero_llm_table)
cursor.execute(create_superhero_llm_hero_2_power_table)
connection.commit()
connection.close()

connection = sqlite3.connect(db_path)
cursor = connection.cursor()
cursor.executemany(superhero_llm_insert_records, data1)
cursor.executemany(llm_hero_2_power_insert_records, data2)
connection.commit()
connection.close()

file_path = './../beyond-database-questions/superheroQueries.csv'
df = pd.read_csv(file_path, header=None)
correct = 30
for sql1, sql2 in zip(df.iloc[:, 3], df.iloc[:, 5]):
    match, exception = evalfunc(sql1, sql2, db_path, db_path)
    if not match:
        correct -= 1
print(correct, correct/30)

7 0.23333333333333334


In [8]:
total_cells = 0
correct_cells = 0
database = 'superhero'
db_path = f'./../databases/dev_databases/{database}/{database}.sqlite'
index = 0
for entry in execute('''
SELECT 
    T1.superhero_name,
    T1.full_name,
    T2.colour AS eye_color,
    T3.colour AS hair_color,
    T4.colour AS skin_color,
    P.publisher_name,
    R.race,
    G.gender,
    A.alignment AS moral_alignment,
    GROUP_CONCAT(SP.power_name, ',') AS powers
FROM 
    superhero AS T1
    INNER JOIN colour AS T2 ON T1.eye_colour_id = T2.id
    INNER JOIN colour AS T3 ON T1.hair_colour_id = T3.id
    INNER JOIN colour AS T4 ON T1.skin_colour_id = T4.id
    INNER JOIN publisher AS P ON T1.publisher_id = P.id
    INNER JOIN race AS R ON T1.race_id = R.id
    INNER JOIN gender AS G ON T1.gender_id = G.id
    INNER JOIN alignment AS A ON T1.alignment_id = A.id
    LEFT JOIN hero_power AS HP ON T1.id = HP.hero_id
    LEFT JOIN superpower AS SP ON HP.power_id = SP.id
GROUP BY 
    T1.id, T1.superhero_name, T1.full_name, T2.colour, T3.colour, T4.colour, 
    P.publisher_name, R.race, G.gender, A.alignment
        ''', db_path):
    superhero_name = entry[0]
    full_name = entry[1]
    identifier = 'superhero-'+str(index)
    power_set = None
    if entry[-1] == None:
        power_set = set()
    else:
        power_set = set(entry[-1].split(','))
    if identifier not in identifier_2_hero_record:
        cells = len(entry) - 1 + len(power_list)
        total_cells += cells
        continue
    record = identifier_2_hero_record[identifier]
    
    if len(entry) == 10:
        generated_power_set = set(record[-1].split(','))
    else:
        generated_power_set = set()
        for power_name in entry[9:]:
            generated_power_set.add(power_name)
        
    for x, y in zip(entry[:-1], record[:-1]):
        total_cells += 1
        if x == y:
            correct_cells += 1
    
    total_cells += len(power_set)
    F1 = 0
    if not generated_power_set or not power_set:
        F1 = 0
    else:
        intersection = len(power_set.intersection(generated_power_set))
        precision = 1.0 * intersection/len(power_set)
        recall = 1.0 * intersection/len(generated_power_set)
        if precision + recall == 0:
            F1 = 0
        else:
            F1 = 2 * precision * recall / (precision + recall)
    correct_cells += F1

    index += 1
    
print('correct cells: ', correct_cells)
print('total cells: ', total_cells)

correct cells:  5339.750490036954
total cells:  11712


# Formula_1

In [9]:
from SQLs import create_formula_1_llm_circuits, create_formula_1_llm_people, create_formula_1_llm_races,\
    llm_circuits_insert_records, llm_people_insert_records, llm_races_insert_records 

database = 'formula_1'
db_path = f'./../databases/dev_databases/{database}/{database}.sqlite'
connection = sqlite3.connect(db_path)
cursor = connection.cursor()
cursor.execute('DROP TABLE IF EXISTS llm_circuits;')
cursor.execute('DROP TABLE IF EXISTS llm_people;')
cursor.execute('DROP TABLE IF EXISTS llm_races;')
connection.commit()
connection.close()

connection = sqlite3.connect(db_path)
cursor = connection.cursor()
cursor.execute(create_formula_1_llm_circuits)
cursor.execute(create_formula_1_llm_people)
cursor.execute(create_formula_1_llm_races)
connection.commit()
connection.close()

# circuits
data1 = []
# constructors and drivers
data2 = []
# races
data3 = []


# modify
output_name = 'llm_generations/5-shots/formula_1_gpt4turbo_5shot.jsonl'



identifier_2_fomulra_1_record = {}
rows = []
with open(output_name, 'r') as outfile:
    for line in outfile:
        response = json.loads(line)
        rid = response['custom_id']
        result = response['response']['body']["choices"][0]['message']['content']
        result = result.replace(", '", ",'")
        data_io = StringIO(result)
        reader = csv.reader(data_io, delimiter=',', quotechar="'")
        entry = next(reader)
        entry = [e.strip() for e in entry]
        if 'circuit' in rid:
            if len(entry) == 4: # missing keys:
                entry = prefix[rid] + entry
            if len(entry) != 6: 
                continue
            for i, key in enumerate(prefix_keys[rid]):
                entry[i] = key
            data1.append(entry)
        elif 'constructor' in rid:
            if len(entry) == 2: # missing keys and null values
                entry = prefix[rid] + ['NULL', 'NULL'] + entry
            if len(entry) != 5: 
                continue
            for i, key in enumerate(prefix_keys[rid]):
                entry[i] = key
            data2.append(entry)
        elif 'driver' in rid:
            if len(entry) == 4: # missing keys
                entry = prefix[rid] + entry
            elif len(entry) == 6:
                # perhaps dob seperated into 2 fields
                dob = entry[1]+' '+entry[2]
                entry = [entry[0], dob] +entry[3:]
            if len(entry) != 5: 
                continue
            for i, key in enumerate(prefix_keys[rid]):
                entry[i] = key
            data2.append(entry)
        else:
            if len(entry) == 2: # missing keys
                entry = prefix[rid] + entry
            if len(entry) != 4: 
                continue
            for i, key in enumerate(prefix_keys[rid]):
                entry[i] = key
            data3.append(entry)
        identifier_2_fomulra_1_record[rid] = entry
        
connection = sqlite3.connect(db_path)
cursor = connection.cursor()
cursor.executemany(llm_circuits_insert_records, data1)
cursor.executemany(llm_people_insert_records, data2)
cursor.executemany(llm_races_insert_records , data3)
connection.commit()
connection.close()

database = 'formula_1'
file_path = './../beyond-database-questions/formula_1Queries.csv'
df = pd.read_csv(file_path, header=None)
correct = 30
for sql1, sql2 in zip(df.iloc[:, 3], df.iloc[:, 5]):
    match, exception = evalfunc(sql1, sql2, db_path, db_path)
    if not match and not exception:
        correct -= 1
print(correct, correct/30)


15 0.5


In [10]:
correct_cells = 0
total_cells = 0
database = 'formula_1'
db_path = f'./../databases/dev_databases/{database}/{database}.sqlite'

index = 0
for entry in execute('''
SELECT location, name, country, url, lat, lng
    FROM circuits
        ''', db_path):
    location = entry[0]
    name = entry[1]
    identifier = 'circuits-'+str(index)
    total_cells += 4
    if identifier not in identifier_2_fomulra_1_record:
        continue
    record = identifier_2_fomulra_1_record[identifier]
    for x, y in zip(entry[2:], record[2:]):
        if x == y:
            correct_cells += 1
    index += 1

index = 0
for entry in execute('''
SELECT name AS name, NULL AS dob, NULL AS code, nationality AS nation, url AS person_url
    FROM constructors
        ''', db_path):
    name = entry[0]
    dob = 'NULL'
    code = 'NULL'
    identifier = 'constructor-'+str(index)
    
    total_cells += 2
    if identifier not in identifier_2_fomulra_1_record:
        continue
    record = identifier_2_fomulra_1_record[identifier]
    for x, y in zip(entry[3:], record[3:]):
        if x == y:
            correct_cells += 1
    index += 1

index = 0
for entry in execute('''
SELECT forename || surname AS name,  dob AS dob, code AS code, nationality AS nation, url AS person_url
    FROM drivers
        ''', db_path):
    name = entry[0]
    identifier = 'driver-'+str(index)
    total_cells += 4
    if identifier not in identifier_2_fomulra_1_record:
        continue
    record = identifier_2_fomulra_1_record[identifier]
    for x, y in zip(entry[1:], record[1:]):
        if x == y:
            correct_cells += 1
    index += 1
            
index = 0
for entry in execute('''
SELECT T1.name, T1.year, T1.date, T2.url
    FROM races AS T1 INNER JOIN seasons AS T2 ON T2.year = T1.year 
        ''', db_path):
    race = entry[0]
    year = entry[1]
    identifier = 'races-'+str(index)
    total_cells += 2
    if identifier not in identifier_2_fomulra_1_record:
        continue
    record = identifier_2_fomulra_1_record[identifier]
    for x, y in zip(entry[2:], record[2:]):
        if x == y:
            correct_cells += 1
    index += 1
    
print('correct cells: ', correct_cells)
print('total cells: ', total_cells)

correct cells:  4521
total cells:  6016


# European Football 2

In [11]:
from SQLs import create_football_llm_match, create_football_llm_team, create_football_llm_player,\
    llm_match_insert_records, llm_team_insert_records, llm_player_insert_records 
database = 'european_football_2'
db_path = f'./../databases/dev_databases/{database}/{database}.sqlite'
connection = sqlite3.connect(db_path)
cursor = connection.cursor()
cursor.execute('DROP TABLE IF EXISTS llm_match;')
cursor.execute('DROP TABLE IF EXISTS llm_team;')
cursor.execute('DROP TABLE IF EXISTS llm_player;')
connection.commit()
connection.close()

connection = sqlite3.connect(db_path)
cursor = connection.cursor()
cursor.execute(create_football_llm_match)
cursor.execute(create_football_llm_team)
cursor.execute( create_football_llm_player)
connection.commit()
connection.close()


# llm_match
data1 = []
# llm_team
data2 = []
# llm_player
data3 = []

# modify
output_name = 'llm_generations/5-shots/european_football_2_gpt4turbo_5shot.jsonl'



identifier_2_football_record = {}
rows = []
with open(output_name, 'r') as outfile:
    for line in outfile:
        response = json.loads(line)
        rid = response['custom_id']
        result = response['response']['body']["choices"][0]['message']['content']
        result = result.replace(", '", ",'")
        data_io = StringIO(result)
        reader = csv.reader(data_io, delimiter=',', quotechar="'")
        entry = next(reader)
        entry = [e.strip() for e in entry]
        if 'match' in rid:
            if len(entry) == 4:
                entry = prefix_keys[rid] + entry
            elif len(entry) == 8:
                initial = entry[0] + entry[1]
                entry = [initial] + entry[2:]
            if len(entry) != 7:
                print(rid, entry)
                continue
            for i, key in enumerate(prefix_keys[rid]):
                entry[i] = key
            data1.append(entry)
        elif 'team' in rid:
            # fix wrong spell in rid: footbal -> football
            assert 'footbal' in rid
            rid = 'football_' + rid.split('_')[1]
            if len(entry) == 1:
                entry = prefix_keys[rid] + entry
            if len(entry) != 2:
                continue
            for i, key in enumerate(prefix_keys[rid]):
                entry[i] = key
            data2.append(entry)
        else:
            if len(entry) == 3:
                entry = prefix_keys[rid] + entry
            if len(entry) != 5:
                continue
            try:
                entry[-1] = float(entry[-1])
            except:
                continue
            for i, key in enumerate(prefix_keys[rid]):
                entry[i] = key
            data3.append(entry)
        identifier_2_football_record[rid] = entry

connection = sqlite3.connect(db_path)
cursor = connection.cursor()
cursor.executemany(llm_match_insert_records, data1)
cursor.executemany(llm_team_insert_records, data2)
cursor.executemany(llm_player_insert_records, data3)
connection.commit()
connection.close()



database = 'european_football_2'
file_path = './../beyond-database-questions/european_football_2Queries.csv'
df = pd.read_csv(file_path, header=None)
correct = 30
for sql1, sql2 in zip(df.iloc[:, 3], df.iloc[:, 5]):
    match, exception = evalfunc(sql1, sql2, db_path, db_path)
    if not match and not exception:
        correct -= 1
print(correct, correct/30)


9 0.3


In [12]:
correct_cells = 0
total_cells = 0
database = 'european_football_2'
db_path = f'./../databases/dev_databases/{database}/{database}.sqlite'

index = 0
for entry in execute('''
SELECT hometeam.team_long_name,
    awayteam.team_long_name,
    t1.date,
    t2.name,
    t1.home_team_goal,
    t1.away_team_goal,
    t3.name
    FROM Match AS t1 INNER JOIN Team AS awayteam ON t1.away_team_api_id = awayteam.team_api_id 
    INNER JOIN Team AS hometeam ON t1.home_team_api_id = hometeam.team_api_id 
    INNER JOIN League AS t2 ON t1.league_id = t2.id
    INNER JOIN Country AS t3 ON t1.country_id = t3.id
        ''', db_path):
    home_team = entry[0]
    away_team = entry[1]
    date = entry[2]
    identifier = 'football_matches-'+str(index)
    
    total_cells += 4
    if identifier not in identifier_2_football_record:
        continue
    record = identifier_2_football_record[identifier]
    for x, y in zip(entry[3:], record[3:]):
        if x == y:
            correct_cells += 1
    index += 1
    
index = 0
for entry in execute('''
SELECT DISTINCT team_long_name, team_short_name FROM Team
        ''', db_path):
    team_long_name = entry[0]
    identifier = 'footbal_team-'+str(index)
    
    total_cells += 1
    if identifier not in identifier_2_football_record:
        continue
    record = identifier_2_football_record[identifier]
    for x, y in zip(entry[1:], record[1:]):
        if x == y:
            correct_cells += 1
    
    index += 1

index = 0
for entry in execute('''
SELECT DISTINCT T1.player_name, T1.weight, T1.birthday, T2.preferred_foot, T1.height
    FROM Player AS T1 INNER JOIN Player_Attributes AS t2 ON T1.player_api_id = t2.player_api_id
        ''', db_path):
    player_name = entry[0]
    weight = entry[1]
    identifier = 'football_players-'+str(index)
    
    total_cells += 3
    if identifier not in identifier_2_football_record:
        continue
    record = identifier_2_football_record[identifier]
    for x, y in zip(entry[2:], record[2:]):
        if x == y:
            correct_cells += 1
    
    index += 1

print('correct cells: ', correct_cells)
print('total cells: ', total_cells)

correct cells:  63403
total cells:  131863
